In [1]:
import os
import glob

# from src.envs import TradingEnv
from src.envs.environment import TradingEnv
from src.utils import DataLoader, Preprocessor, backtest

Loading BokehJS ...

In [2]:
if len(glob.glob(f"./data/^N225/*.csv")) != 0:
    data = DataLoader.load_data(f"./data/^N225/ohlcv.csv")
    features = DataLoader.load_data(f"./data/^N225/features.csv")
else:
    os.makedirs(f"./data/^N225", exist_ok=True)
    data = DataLoader.fetch_data("^N225", interval="1d")
    data, features = Preprocessor.extract_features(data)
    data.to_csv(f"./data/^N225/ohlcv.csv")
    features.to_csv(f"./data/^N225/features.csv")

data = data.iloc[-50:]
features = features.iloc[-50:]

data_train, features_train, data_eval, features_eval = Preprocessor.train_test_split(data, features, test_size=0.5)

In [3]:
from src.envs.actions import BuySell
env = TradingEnv(data_eval, features_eval, debug=True, stop_loss=True)

In [4]:
obs = env.reset()
done = False
while not done:
    obs, _, done, _ = env.step(env.action_space.sample())
    # obs, _, done, _ = env.step(0)
    print(obs[:, -4:])
    for order in env.orders:
        print(env.position)
        print(order.stop)
        # print(order.stop_price)

=============== Environment (2021-10-19 00:00:00) ===============
Price: 29215.51953125
Assets: 100000
Equity: 100000
Orders: [<Order size=3.0, sl=27713.52633, contingent=0>]
Trades: []
Position: <Position: 0 (0 trades)>
Closed Trades: []
Action: 2, Reward: 0, Done: False

[0.00036984 0.         0.05564158]
[[-1.69431630e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.60370038e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.45267385e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.82520596e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.32862773e+00  3.69835133e-04  0.00000000e+00  5.56415822e-02]]
<Position: 3 (1 trades)>
27713.526328125
=============== Environment (2021-10-20 00:00:00) ===============
Price: 29255.55078125
Assets: 100000
Equity: 100032.44719140626
Orders: [<Order size=-3.0, stop=27713.52633, contingent=1>]
Trades: [<Trade size=3 time=5- price=29244.735050781248- pnl=32>]
Position: <Position: 3 (1 trades)>
Closed Trades: []
Actio

In [5]:
# backtest(env)
backtest(env, agent="Random")
# results = backtest(env, agent="Buy&Hold", save_dir="test", plot=True)
# results = backtest(env, agent="Sell&Hold", save_dir="test", plot=True)
# results

=============== Environment (2021-10-19 00:00:00) ===============
Price: 29215.51953125
Assets: 100000
Equity: 100000
Orders: []
Trades: []
Position: <Position: 0 (0 trades)>
Closed Trades: []
Action: 0, Reward: 0, Done: False

[0 0 0]
=============== Environment (2021-10-20 00:00:00) ===============
Price: 29255.55078125
Assets: 100000
Equity: 100000
Orders: []
Trades: []
Position: <Position: 0 (0 trades)>
Closed Trades: []
Action: 0, Reward: 0.0, Done: False

[0 0 0]
=============== Environment (2021-10-21 00:00:00) ===============
Price: 28708.580078125
Assets: 100000
Equity: 100000
Orders: []
Trades: []
Position: <Position: 0 (0 trades)>
Closed Trades: []
Action: 0, Reward: 0.0, Done: False

[0 0 0]
=============== Environment (2021-10-22 00:00:00) ===============
Price: 28804.849609375
Assets: 100000
Equity: 100000
Orders: [<Order size=-3.0, sl=29784.00047, contingent=0>]
Trades: []
Position: <Position: 0 (0 trades)>
Closed Trades: []
Action: 1, Reward: 0.0, Done: False

[0.      

Start                     2021-10-12 00:00:00
End                       2021-11-16 00:00:00
Duration                     35 days 00:00:00
Exposure Time [%]                        60.0
Equity Final [$]                102387.288584
Equity Peak [$]                 103471.456146
Return [%]                           2.387289
Buy & Hold Return [%]                5.587941
Return (Ann.) [%]                   26.846972
Volatility (Ann.) [%]                9.479971
Sharpe Ratio                         2.659512
Sortino Ratio                        5.735601
Calmar Ratio                         25.62238
Max. Drawdown [%]                   -1.047794
Avg. Drawdown [%]                   -0.563646
Max. Drawdown Duration        8 days 00:00:00
Avg. Drawdown Duration        5 days 00:00:00
# Trades                                    8
Win Rate [%]                             50.0
Best Trade [%]                         2.5085
Worst Trade [%]                     -1.235579
Avg. Trade [%]                    